In [1]:
import jieba
import jieba.analyse
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import json
from pypinyin import lazy_pinyin

In [16]:
# 文件位置
joblist_path = '../../../demo2/Q2_jobList.xlsx'
resume_path = '../../../demo2/Q2_resume.xlsx'

joblist_save_path = '../../../demo2/Q2_joblist(分词).xlsx'
resume_save_path = '../../../demo2/Q2_resume(分词).xlsx'

joblist_save_tolower_path = '../../../demo3/Q3_joblist.xlsx'
resume_save_tolower_path = '../../../demo3/Q3_resume.xlsx'

common_stopwords_path = 'stopwords/common_stopwords.txt'
special_stopwords_path = 'stopwords/原数据中的停用词.txt'
stopwords_path = 'stopwords/stop_words.txt'

employee_dict_path = 'dict/employee_dict.txt'
dict_path = 'dict/dict.txt'

In [3]:
# 停用词
common_stopwords = open(common_stopwords_path,'r')
special_stopwords = open(special_stopwords_path,'r')

stopwords_common = []
stopwords_special = []
stop_words_list=[]

for i in common_stopwords.readlines():
    stopwords_common.append(i.strip())
    stop_words_list.append(i.strip())
for i in special_stopwords.readlines():
    stopwords_special.append(i.strip())
    stop_words_list.append(i.strip())
with open(stopwords_path, 'w') as f:
    for word in stop_words_list:
        f.write(word+'\n')

In [4]:
# 获取自定义词典
# # 根据chatgpt得到分词后的文本-->(employee_dict)
# employee_clean = pd.read_excel('emp_clean.xlsx')
# employee_clean = employee_clean.replace(np.NAN,'')
# employee_clean = employee_clean.replace('、',',', regex=True)
# employee_dict = set()
# for i in employee_clean.columns[1:-1]:
#     for j in range(employee_clean.shape[0]):
#         if employee_clean.loc[j,i]:
#             for word in employee_clean.loc[j,i].split(','):
#                 if word:
#                     employee_dict.add(word)
# employee_dict = list(employee_dict)
# employee_dict.sort(key=lambda x: lazy_pinyin(x)[0][0])

# # 将集合中的元素写入文本文件中
# with open('dict/employee_dict.txt', 'w') as f:
#     for word in employee_dict:
#         if word:
#             f.write(word + ' 10000'+ '\n')

# 人工过滤停用词-->(employee_dict_list)
employee_dict = open(employee_dict_path,'r')
employee_dict_list = [word.strip() for word in employee_dict.readlines()]
employee_dict_list = [x for x in employee_dict_list if x != ""]
employee_dict_list.sort(key=lambda x: lazy_pinyin(x)[0][0])
with open(dict_path, 'w') as f:
    for word in employee_dict_list:
        if word:
            f.write(word + '\n')

In [5]:
#清理文本中常见的六种特殊符号[,],【,】,',",",“,”
def clean_data(data):
    special_txt = r'\[|\]|【|】|\'|"|“|”'
    data = data.replace(special_txt,'',regex=True)
    data = data.replace('无','')
    data = data.replace('未填写','')
    data = data.replace(np.nan,'')
    data = data.replace('无,无,无','')  
    return data

#分割列
def split_column(data, column, split_columns_list):
    a = 0
    for _ in split_columns_list:
        data[_] = data[column].map(lambda x:x.split(',')[a] if (x != '') else '')
        a += 1
    return data

# 根据列、起始时间、新列名称计算时间间隔
def time_count_days(data, row, start_time_column, end_time_column, new_column):
    if data.loc[row, start_time_column] == '' or data.loc[row, end_time_column] == '':
        return
    else: 
        day = 0
        for i in range(len(data.loc[row, end_time_column].split(','))):
            a = datetime.strptime(data.loc[row, start_time_column].split(',')[i],"%Y-%m-%d")
            b = datetime.strptime(data.loc[row, end_time_column].split(',')[i],"%Y-%m-%d")
            day += (b-a).days
        data.loc[row, new_column] = day

# 对resume文件进行分词

In [6]:
resume_inf = pd.read_excel(resume_path, dtype = {'id':str})     #应聘人员信息

resume_inf['id'] = resume_inf['id'].apply(lambda x:x.strip()) # 在jupyter上，\t不一定代表制表符，也有可能是字符串
resume_inf = clean_data(resume_inf)

resume_jieba = ['selfEvaluation','workEP_descriptions','projectEp_descriptions','projectEP_achievements','trainEP_description']
resume_jieba_newname = ['selfEvaluation_keywords','workEP_descriptions_keywords','projectEp_descriptions_keywords','projectEP_achievements_keywords','trainEP_description_keywords']

jieba.load_userdict(dict_path) 
jieba.analyse.set_stop_words(stopwords_path)

for i in range(len(resume_jieba)):
    resume_inf[resume_jieba[i]] = resume_inf[resume_jieba[i]].map(lambda x:'' if x in stop_words_list else x)
    resume_inf[resume_jieba_newname[i]] = resume_inf[resume_jieba[i]].map(lambda x:jieba.analyse.extract_tags(x,topK=20, allowPOS=('n|a|d|q|x')))
    resume_inf[resume_jieba_newname[i]] = resume_inf[resume_jieba_newname[i]].astype(str)

# 修改生日格式
resume_inf['birthday'] = resume_inf['birthday'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.isna(x) != True else '') #会自动将x转成str类型

# 计算年龄
today = date.today()
resume_inf['age'] = resume_inf['birthday'].apply(lambda x: today.year - datetime.strptime(x,'%Y-%m-%d').year - ((today.month, today.day) < (datetime.strptime(x,'%Y-%m-%d').month, datetime.strptime(x,'%Y-%m-%d').day)) if x != '' else x)
resume_inf['age'] = resume_inf['age'].apply(lambda x : str(x)+'岁')

# 计算工作年限
# 发现出现一个异常数据，在第115列，编号为1471800130456911872的廖女士的工作结束日期为2020年8月，早于开始日期2021年7月，因此认为是异常数据，将其手动改为2021年8月
worktime_day = ['workEp_start','workEp_end']
projecttime_day = ['projectEP_start','projectEP_end']
traintime_day = ['trainEP_start','trainEP_end']
edutime_day = ['eduEp_start','eduEp_end']
days = ['worktime_days', 'projecttime_days', 'traintime_days', 'edutime_days']
resume_inf.loc[115,worktime_day[1]] = '2021-08-27,2019-08-15'
for i in range(resume_inf.shape[0]):
    time_count_days(resume_inf, i, worktime_day[0], worktime_day[1], days[0])
    time_count_days(resume_inf, i, projecttime_day[0], projecttime_day[1], days[1])
    time_count_days(resume_inf, i, traintime_day[0], traintime_day[1], days[2])
    time_count_days(resume_inf, i, edutime_day[0], edutime_day[1], days[3])
resume_inf = clean_data(resume_inf) # 将np.nan转成''

# 将时间转成int类型
for i in days:
    resume_inf[i] = resume_inf[i].apply(lambda x : int(x) if x != '' else x)

# 计算是否应届
resume_inf['is_graduate'] = resume_inf['eduEp_end'].apply(lambda x: '应届' if '2023'in x else '非应届' if len(x) != 0 else '未知')

# 计算最高学历
# 定义学历列表
education_list = ['大专', '本科', '硕士', '博士']
def highest_education(x):
    if x == '':
        return ''
    else:
        # 定义最高学历和最高学历索引
        highest_education = ''
        highest_index = -1
        split_education = x.split(',')
        # 遍历学历列表，查找最高学历
        for edu in split_education:
            # 查找学历在学历列表中的索引
            index = education_list.index(edu)
            if index > highest_index:
                highest_index = index
                highest_education = edu
        return highest_education
resume_inf['highest_education'] = resume_inf['eduEp_educationBackgrounds'].apply(highest_education)
# 分割省市区
city_split = ['province', 'county', 'region']
resume_inf = split_column(resume_inf, 'expectCity', city_split)

resume_inf = clean_data(resume_inf)
# 导出数据集
resume_inf.to_excel(resume_save_path,index=False)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/c3/vyxk7z2n5dx_dgz2fc4gsz440000gn/T/jieba.cache
Loading model cost 0.542 seconds.
Prefix dict has been built successfully.


In [17]:
resume_lower_column = ['expectPosition', 'trainEP_recordName', 'workEp_positionNames', 'keyword_info', 'lang_info', 'cert_info', 'skill_info', 'concatenated', 'selfEvaluation_keywords','workEP_descriptions_keywords','projectEp_descriptions_keywords','projectEP_achievements_keywords','trainEP_description_keywords']
for i in resume_lower_column:
    resume_inf[i] = resume_inf[i].str.lower()
resume_inf.to_excel(resume_save_tolower_path,index=False)

# 对joblist_inf进行分词

In [7]:
joblist_inf = pd.read_excel(joblist_path, dtype = {'id':str})
joblist_inf['jobRequiredments_keywords'] = joblist_inf['jobRequiredments'].map(lambda x:jieba.analyse.extract_tags(x))
joblist_inf['id'] = joblist_inf['id'].apply(lambda x:x.strip()) # 在jupyter上，\t不一定代表制表符，也有可能是字符串
joblist_inf.to_excel(joblist_save_path,index=False)
joblist_inf = pd.read_excel(joblist_save_path, dtype = {'id':str})
joblist_inf = clean_data(joblist_inf)
joblist_inf.to_excel(joblist_save_path,index=False)

出现了bug，只有先保存joblist，再读取joblist，才能对这个df里面的特殊符号进行去除。

In [19]:
joblist_lower_column = ['positionName', 'keyword_info', 'skill_info', 'concatenated', 'jobRequiredments_keywords']
for i in joblist_lower_column:
    joblist_inf[i] = joblist_inf[i].str.lower()
joblist_inf.to_excel(joblist_save_tolower_path,index=False)